In [ ]:
!pip install -r requirements.txt

In [ ]:
#Step 1
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from openpyxl import load_workbook, Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font
from openpyxl.styles import NamedStyle

# Decorator to measure execution time
def time_complexity(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' took {end_time - start_time:.2f} seconds to execute.")
        return result
    return wrapper

column_mapping = {
    "fulfillment_id": "orderId",
    "ref_id": "refId",
    "index": "index",
    "name": "name",
    "dimensions": "dimensions",
    "cost": "Price",
    "base_cost": "base_cost",
    "total_volume": "Carton_volume",
    "net_volume": "Order_volume",
    "volume_utilization": "volume_utilization",
    "surface_area": "surface_area",
    "total_weight": "total_weight",
    "net_weight": "net_weight",
    "tare_weight": "tare_weight",
    "weight_utilization": "weight_utilization",
    "item_count": "item_count",
    "dim_weight": "dim_weight",
}

# Columns for consolidated output
consolidated_columns = [
    "orderId", "refId", "index", "name", "dimensions", "Price", "base_cost",
    "Carton_volume", "Order_volume", "volume_utilization", "surface_area",
    "total_weight", "net_weight", "tare_weight", "weight_utilization",
    "dim_weight", "item_count", "source_flag"
]

# Combined sheet column order
combined_columns = [
    "orderId_baseline", "orderId", "refId_baseline", "refId",
    "index_baseline", "index", "name_baseline", "name",
    "dimensions_baseline", "dimensions", "Price_baseline", "Price", "Price_Diff",
    "base_cost_baseline", "base_cost", "Carton_volume_baseline", "Carton_volume", "Carton_volume_Diff",
    "Order_volume_baseline", "Order_volume", "Order_volume_Diff", "volume_utilization_baseline", "volume_utilization", 
    "volume_utilization_Diff", "surface_area_baseline", "surface_area", "total_weight_baseline", "total_weight", "total_weight_Diff",
    "net_weight_baseline", "net_weight", "tare_weight_baseline", "tare_weight",
    "weight_utilization_baseline", "weight_utilization", "dim_weight_baseline", "dim_weight",
    "item_count_baseline", "item_count", "Item_Diff"
]

 # Columns to drop to save memory
columns_to_drop = ["item_summary"]  


# Refined suffix extraction:
def refine_suffix(filename):
    """Extracts suffix between 'filepath_####.' and next period/end"""
    match = re.search(r"filepath_(\d+)\.(.*?)(?:$|\.)", filename)
    test = re.search(r"transmutr",filename)
    if match:
        return match.group(2)
    else:
        return match.group(1)
    return None

@time_complexity
# Loads a file in chunks, processes it, and returns a DataFrame
def load_and_process_file_in_chunks(file_path, chunk_size=100000):
    try:
        chunks = []
        for chunk in pd.read_csv(
            file_path, delimiter='|', low_memory=False, memory_map=True,
            on_bad_lines='skip', chunksize=chunk_size
        ):
            # Drop unnecessary columns
            if columns_to_drop:
                chunk = chunk.drop(columns=columns_to_drop, errors='ignore')

            # Rename columns based on mapping
            chunk = chunk.rename(columns={col: column_mapping[col] for col in column_mapping if col in chunk.columns})

            # Extract the source_flag using the filename suffix
            suffix = refine_suffix(file_path)
            if suffix:
                chunk["source_flag"] = suffix
            else:
                print(f"Warning: No valid suffix in file {file_path}.")

            # Reindex to ensure all required columns are present
            chunk = chunk.reindex(columns=consolidated_columns, fill_value=None)

            chunks.append(chunk)

        return pd.concat(chunks, ignore_index=True)
    except Exception as e:
        print(f"Failed to load file {file_path}: {e}")
        return pd.DataFrame()

def calculate_consolidated_fields(df):
    df['dimensions'] = df['dimensions'].astype(str).fillna('')
    # Calculate Dimmed
    df['Dimmed'] = np.where(df['dim_weight'] > df['total_weight'], 'Yes', 'No')

    # Calculate Billed Weight
    df['Billed_Weight'] = np.where(df['dim_weight'] > df['total_weight'], np.ceil(df['dim_weight']), np.ceil(df['total_weight'])).astype(int)

    # Billed Over Actual
    df['total_weight'] = np.ceil(df['total_weight'])

    # Then calculate the 'Billed_over_Actual' column
    df['Billed_over_Actual'] = np.where(df['Billed_Weight'] - df['total_weight'] > 0, df['Billed_Weight'] - df['total_weight'], 0)

    # Split dimensions into L, W, H
    dimensions_split = df['dimensions'].str.split(',', expand=True)

    # Validate that the split resulted in exactly three parts
    if dimensions_split.shape[1] != 3:
        print("Warning: 'dimensions' column does not split into exactly three parts (L,W,H). Filling with NaN.")
        dimensions_split = dimensions_split.reindex(columns=[0,1,2], fill_value=np.nan)

    # Assign to new columns
    df['L'] = pd.to_numeric(dimensions_split[0].str.strip(), errors='coerce').fillna(0)
    df['W'] = pd.to_numeric(dimensions_split[1].str.strip(), errors='coerce').fillna(0)
    df['H'] = pd.to_numeric(dimensions_split[2].str.strip(), errors='coerce').fillna(0)

    

    return df

@time_complexity
def compute_differences(df):
    try:
        # Define the column pairs for differences
        column_pairs = [
            ("Price_baseline", "Price", "Price_Diff"),
            ("Order_volume_baseline", "Order_volume", "Order_volume_Diff"),
            ("item_count_baseline", "item_count", "Item_Diff"),
            ("Carton_volume_baseline", "Carton_volume", "Carton_volume_Diff"),
            ("volume_utilization_baseline", "volume_utilization", "volume_utilization_Diff"),
            ("total_weight_baseline", "total_weight", "total_weight_Diff"),
        ]

        for col_baseline, col, col_diff in tqdm(
            column_pairs, desc="Computing Differences", unit="file", colour="green"
        ):
            if col_baseline in df.columns and col in df.columns:
                df[col_baseline] = pd.to_numeric(df[col_baseline], errors="coerce").fillna(0)
                df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)
                df[col_diff] = df[col] - df[col_baseline]
            else:
                print(f"Skipping difference calculation for {col_diff}: Missing {col_baseline} or {col}")

        return df
    except Exception as e:
        print(f"Error in compute_differences: {e}")
        return df

def apply_dynamic_column_formats(sheet):
    # Define the number style with 2 decimal places for numeric columns
    number_style = NamedStyle(name="number")
    number_style.number_format = '0.00'  # Number format for 2 decimal places

    # Define the percentage style for utilization columns
    percentage_style = NamedStyle(name="percentage")
    percentage_style.number_format = '0.00%'  # Percentage format for utilization columns

    # List of columns to apply number formatting (Price, Carton Volume, etc.)
    numeric_columns = [
        "Price_baseline", "Price", "Price_Diff",
        "Carton_volume_baseline", "Carton_volume", "Carton_volume_Diff",
        "Order_volume_baseline", "Order_volume", "Order_volume_Diff",
        "surface_area_baseline", "surface_area",
        "dim_weight_baseline", "dim_weight"
    ]

    # List of columns for utilization percentage formatting
    utilization_columns = [
        "volume_utilization_baseline", "volume_utilization", "volume_utilization_Diff",
        "weight_utilization_baseline", "weight_utilization"
    ]

    # Loop through columns and apply styles based on column name
    for col in sheet.columns:
        column_letter = col[0].column_letter
        column_name = str(col[0].value).strip() if col[0].value else ""

        # Apply number format to numeric columns
        if column_name in numeric_columns:
            for cell in col:
                cell.number_format = '0.00'  # Apply number format with 2 decimal places

        # Apply percentage format to utilization columns
        elif column_name in utilization_columns:
            for cell in col:
                cell.number_format = '0.00%'  # Apply percentage format

@time_complexity
def process_files(directory, output_directory):
    data_frames = {}
    consolidated_data = []
    files = []
    '''for root, _, filenames in os.walk(directory):
        for filename in filenames:
            base_name = os.path.basename(filename)
            if not any(excluded in base_name for excluded in exclude_files):  # ✅ Exclude unwanted files
                files.append(os.path.relpath(os.path.join(root, filename), directory))  # ✅ Fix double "Upload_Path"'''

    if not files:
        print("No matching files found for processing.")
        return

    # ✅ Ensure a valid baseline file exists
    baseline_files = [f for f in files if "baseline" in os.path.basename(f).lower()]
    if not baseline_files:
        raise ValueError("Baseline file not found. Ensure a file containing 'baseline' in its name exists.")
    baseline_file = baseline_files[0]  # Take the first matching file

    print(f"Processing {len(files)} files (excluding preflight, perfect, all-candidates).")

    # Process files
    with tqdm(total=len(files), desc="Loading files", unit="file", colour="blue") as pbar:
        for filename in files:
            file_path = os.path.join(directory, filename)
            df = load_and_process_file_in_chunks(file_path, chunk_size=100000)
            if not df.empty:
                price_columns = ("Price_baseline", "Price", "Price_Diff")
                for col in price_columns:
                    if col in df.columns:
                        # Convert column to numeric , then multiply by 100
                        df[col] = pd.to_numeric(df[col], errors='coerce') / 100
                data_frames[filename] = df
                consolidated_data.append(df)
            else:
                print(f"Warning: No valid data in file {filename}")
            pbar.update(n=1)


    # Consolidate all data into a single DataFrame
    if consolidated_data:
        consolidated_output = pd.concat(consolidated_data, ignore_index=True)
        try:
            # Perform any calculations
            consolidated_output = calculate_consolidated_fields(consolidated_output)

            # Save the consolidated output
            consolidated_output_path = os.path.join(output_directory, "consolidated_output.xlsx")
            with pd.ExcelWriter(consolidated_output_path, engine='openpyxl') as writer:
                consolidated_output.to_excel(writer, sheet_name="Consolidated_Output", index=False)

                workbook = writer.book
                sheet = writer.sheets["Consolidated_Output"]


                # Header formating
                for cell in sheet[1]:
                    cell.font = Font(bold=False)
                # column formatting
                for col in sheet.columns:
                    column_letter = col[0].column_letter
                    # Get the maximum length of the column header and its values
                    header_length = len(str(col[0].value)) if col[0].value else 0
                    # Calculate the column width
                    column_width = max(header_length + 0.5, 13)
                    # Set the column width
                    sheet.column_dimensions[column_letter].width = column_width

            print(f"Consolidated output saved at: {consolidated_output_path}")

        except Exception as e:
            print(f"Error during consolidated calculations or saving: {e}")
    else:
        print("No data to consolidate.")
    # Define the new output file name
    comparison_filename = "combined_output.xlsx"
    comparison_path = os.path.join(output_directory, comparison_filename)

    # 🚀 Before attempting to pop, ensure baseline_file exists in data_frames
    if baseline_file not in data_frames:
        print(f"❌ ERROR: Baseline file '{baseline_file}' was NOT found in processed data!")
        print(f"🔹 Available keys in data_frames: {list(data_frames.keys())}")
        print(f"🔍 Check if '{baseline_file}' exists in matching_files: {baseline_file in matching_files}")
        return  # 🚨 Stop execution safely

    # 🚀 Now, pop the baseline file safely
    baseline_df = data_frames.pop(baseline_file)

    # 🚀 Debugging Print to Confirm It's Loaded
    print(f"✅ Successfully loaded baseline file: {baseline_file}")


    # Save comparison outputs to Excel
    #baseline_df = data_frames.pop(baseline_file)
    with pd.ExcelWriter(comparison_path, engine='openpyxl') as writer:
        for key, df in data_frames.items():
            sheet_name = refine_suffix(key)  # Using the refined filename as sheet name
            if not sheet_name:
                print(f"Skipping sheet for file {key} due to invalid suffix.")
                continue  # Skip empty suffix

            # Align comparison DataFrame columns with combined_columns
            comparison_df = df.reindex(columns=[col.replace("_baseline", "") for col in combined_columns if "_baseline" not in col])

            # Combine baseline and comparison data
            combined_df = pd.concat(
                [baseline_df.add_suffix("_baseline"), comparison_df],
                axis=1
            ).reindex(columns=combined_columns)

            # Compute differences
            combined_df = compute_differences(combined_df)

            # Keep the difference columns
            difference_columns = [
                "Price_Diff", "Order_volume_Diff", "Item_Diff",
                "Carton_volume_Diff", "volume_utilization_Diff", "total_weight_Diff"
            ]

            if combined_df.empty:
                print(f"Warning: Sheet {sheet_name} has no data. Skipping.")
                continue

            # Write the combined DataFrame to the corresponding sheet
            combined_df.to_excel(writer, sheet_name=sheet_name, index=False)

            # Apply formulas for Dimmed and Billed Weight
            workbook = writer.book
            sheet = writer.sheets[sheet_name]

            # Apply dynamic formatting
            apply_dynamic_column_formats(sheet)

            # Header formating
            for cell in sheet[1]:
                cell.font = Font(bold=False)

            # Column width adjustments
            for col in sheet.columns:
                column_letter = col[0].column_letter
                header_length = len(str(col[0].value)) if col[0].value else 0
                column_width = max(header_length + 0.5, 13)
                sheet.column_dimensions[column_letter].width = column_width

            headers = [str(cell.value).strip() for cell in sheet[1]]
            dim_weight_col_idx = next((i + 1 for i, h in enumerate(headers) if h.startswith("dim_weight_")), None)
            total_weight_col_idx = next((i + 1 for i, h in enumerate(headers) if h.startswith("total_weight_")), None)

            if dim_weight_col_idx is None or total_weight_col_idx is None:
                print(f"Warning: Required columns for Dimmed and Billed Weight formulas not found in '{sheet_name}'. Headers: {headers}")
                continue

            last_column = len(headers)
            dimmed_col_letter = get_column_letter(last_column + 1)
            billed_weight_col_letter = get_column_letter(last_column + 2)

            sheet[f"{dimmed_col_letter}1"] = "Dimmed"
            sheet[f"{billed_weight_col_letter}1"] = "Billed_Weight"

            for row in range(2, sheet.max_row + 1):
                sheet[f"{dimmed_col_letter}{row}"] = (
                    f"=IF({get_column_letter(dim_weight_col_idx)}{row} > {get_column_letter(total_weight_col_idx)}{row}, \"Yes\", \"No\")"
                )
                sheet[f"{billed_weight_col_letter}{row}"] = (
                    f"=IF({get_column_letter(dim_weight_col_idx)}{row} > {get_column_letter(total_weight_col_idx)}{row}, "
                    f"ROUNDUP({get_column_letter(dim_weight_col_idx)}{row}, 0), ROUNDUP({get_column_letter(total_weight_col_idx)}{row}, 0))"
                )

            final_df = combined_df[difference_columns]

    print(f"Comparison Excel saved at {comparison_path}")

if __name__ == "__main__":
    directory = "Upload_Path"
    output_directory = "Output_Path"
    os.makedirs(output_directory, exist_ok=True)
    process_files(directory, output_directory)

In [ ]:
#Step 1
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from openpyxl import load_workbook, Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font
from openpyxl.styles import NamedStyle

# Decorator to measure execution time
def time_complexity(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' took {end_time - start_time:.2f} seconds to execute.")
        return result
    return wrapper

# Column rename mapping
column_mapping = {
    "fulfillment_id": "orderId",
    "ref_id": "refId",
    "index": "index",
    "name": "name",
    "dimensions": "dimensions",
    "cost": "Price",
    "base_cost": "base_cost",
    "total_volume": "Carton_volume",
    "net_volume": "Order_volume",
    "volume_utilization": "volume_utilization",
    "surface_area": "surface_area",
    "total_weight": "total_weight",
    "net_weight": "net_weight",
    "tare_weight": "tare_weight",
    "weight_utilization": "weight_utilization",
    "item_count": "item_count",
    "dim_weight": "dim_weight",
}

# Columns for consolidated output
consolidated_columns = [
    "orderId", "refId", "index", "name", "dimensions", "Price", "base_cost",
    "Carton_volume", "Order_volume", "volume_utilization", "surface_area",
    "total_weight", "net_weight", "tare_weight", "weight_utilization",
    "dim_weight", "item_count", "source_flag"
]

# Combined sheet column order
combined_columns = [
    "orderId_baseline", "orderId", "refId_baseline", "refId",
    "index_baseline", "index", "name_baseline", "name",
    "dimensions_baseline", "dimensions", "Price_baseline", "Price", "Price_Diff",
    "base_cost_baseline", "base_cost", "Carton_volume_baseline", "Carton_volume", "Carton_volume_Diff",
    "Order_volume_baseline", "Order_volume", "Order_volume_Diff", "volume_utilization_baseline", "volume_utilization", 
    "volume_utilization_Diff", "surface_area_baseline", "surface_area", "total_weight_baseline", "total_weight", "total_weight_Diff",
    "net_weight_baseline", "net_weight", "tare_weight_baseline", "tare_weight",
    "weight_utilization_baseline", "weight_utilization", "dim_weight_baseline", "dim_weight",
    "item_count_baseline", "item_count", "Item_Diff"
]

 # Columns to drop to save memory
columns_to_drop = ["item_summary"]  

# Refined suffix extraction:
def refine_suffix(filename):
    """Extracts suffix between '_####.' and next period/end"""
    match = re.search(r"filepath_(\d+)\.(.*?)(?:$|\.)", filename)
    test = re.search(r"transmutr",filename)
    if match:
        return match.group(2)
    else:
        return match.group(1)
    return None

@time_complexity
# Loads a file in chunks, processes it, and returns a DataFrame
def load_and_process_file_in_chunks(file_path, chunk_size=100000):
    try:
        chunks = []
        for chunk in pd.read_csv(
            file_path, delimiter='|', low_memory=False, memory_map=True,
            on_bad_lines='skip', chunksize=chunk_size
        ):
            # Drop unnecessary columns
            if columns_to_drop:
                chunk = chunk.drop(columns=columns_to_drop, errors='ignore')

            # Rename columns based on mapping
            chunk = chunk.rename(columns={col: column_mapping[col] for col in column_mapping if col in chunk.columns})

            # Extract the source_flag using the filename suffix
            suffix = refine_suffix(file_path)
            if suffix:
                chunk["source_flag"] = suffix
            else:
                print(f"Warning: No valid suffix in file {file_path}.")

            # Reindex to ensure all required columns are present
            chunk = chunk.reindex(columns=consolidated_columns, fill_value=None)

            chunks.append(chunk)

        return pd.concat(chunks, ignore_index=True)
    except Exception as e:
        print(f"Failed to load file {file_path}: {e}")
        return pd.DataFrame()

def calculate_consolidated_fields(df):    
    # Calculate Dimmed
    df['Dimmed'] = np.where(df['dim_weight'] > df['total_weight'], 'Yes', 'No')

    # Calculate Billed Weight
    df['Billed_Weight'] = np.where(df['dim_weight'] > df['total_weight'], np.ceil(df['dim_weight']), np.ceil(df['total_weight'])).astype(int)
    
    # Billed Over Actual
    df['total_weight'] = np.ceil(df['total_weight'])

    # Then calculate the 'Billed_over_Actual' column
    df['Billed_over_Actual'] = np.where(df['Billed_Weight'] - df['total_weight'] > 0, df['Billed_Weight'] - df['total_weight'], 0)

    # Split dimensions into L, W, H
    dimensions_split = df['dimensions'].str.split(',', expand=True)
    
    # Validate that the split resulted in exactly three parts
    #if dimensions_split.shape[1] != 3:
    #    print("Warning: 'dimensions' column does not split into exactly three parts (L,W,H). Filling with NaN.")
    #    dimensions_split = dimensions_split.reindex(columns=[0,1,2], fill_value=np.nan)

    # Assign to new columns
    df['L'] = pd.to_numeric(dimensions_split[0].str.strip(), errors='coerce')
    df['W'] = pd.to_numeric(dimensions_split[1].str.strip(), errors='coerce')
    df['H'] = pd.to_numeric(dimensions_split[2].str.strip(), errors='coerce')

    return df

@time_complexity
def compute_differences(df):
    try:
        # Define the column pairs for differences
        column_pairs = [
            ("Price_baseline", "Price", "Price_Diff"),
            ("Order_volume_baseline", "Order_volume", "Order_volume_Diff"),
            ("item_count_baseline", "item_count", "Item_Diff"),
            ("Carton_volume_baseline", "Carton_volume", "Carton_volume_Diff"),
            ("volume_utilization_baseline", "volume_utilization", "volume_utilization_Diff"),
            ("total_weight_baseline", "total_weight", "total_weight_Diff"),
        ]

        for col_baseline, col, col_diff in tqdm(
            column_pairs, desc="Computing Differences", unit="file", colour="green"
        ):
            if col_baseline in df.columns and col in df.columns:
                df[col_baseline] = pd.to_numeric(df[col_baseline], errors="coerce").fillna(0)
                df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)
                df[col_diff] = df[col] - df[col_baseline]
            else:
                print(f"Skipping difference calculation for {col_diff}: Missing {col_baseline} or {col}")

        return df
    except Exception as e:
        print(f"Error in compute_differences: {e}")
        return df

def apply_dynamic_column_formats(sheet):
    # Define the number style with 2 decimal places for numeric columns
    number_style = NamedStyle(name="number")
    number_style.number_format = '0.00'  # Number format for 2 decimal places
    
    # Define the percentage style for utilization columns
    percentage_style = NamedStyle(name="percentage")
    percentage_style.number_format = '0.00%'  # Percentage format for utilization columns

    # List of columns to apply number formatting (Price, Carton Volume, etc.)
    numeric_columns = [
        "Price_baseline", "Price", "Price_Diff", 
        "Carton_volume_baseline", "Carton_volume", "Carton_volume_Diff", 
        "Order_volume_baseline", "Order_volume", "Order_volume_Diff", 
        "surface_area_baseline", "surface_area", 
        "dim_weight_baseline", "dim_weight"
    ]
    
    # List of columns for utilization percentage formatting
    utilization_columns = [
        "volume_utilization_baseline", "volume_utilization", "volume_utilization_Diff", 
        "weight_utilization_baseline", "weight_utilization"
    ]

    # Loop through columns and apply styles based on column name
    for col in sheet.columns:
        column_letter = col[0].column_letter
        column_name = str(col[0].value).strip() if col[0].value else ""

        # Apply number format to numeric columns
        if column_name in numeric_columns:
            for cell in col:
                cell.number_format = '0.00'  # Apply number format with 2 decimal places
        
        # Apply percentage format to utilization columns
        elif column_name in utilization_columns:
            for cell in col:
                cell.number_format = '0.00%'  # Apply percentage format

@time_complexity
def process_files(directory, output_directory):
    files = os.listdir(directory)
    data_frames = {}
    consolidated_data = []
    baseline_file = None

    # Identify the baseline file
    for filename in files:
        if "baseline" in filename.lower():
            baseline_file = filename
            break

    if not baseline_file:
        raise ValueError("Baseline file not found. Ensure a file containing 'baseline' in its name exists.")

    # Process files
    with tqdm(total=len(files), desc="Loading files", unit="file", colour="blue") as pbar:
        for filename in files:
            file_path = os.path.join(directory, filename)
            df = load_and_process_file_in_chunks(file_path, chunk_size=100000)
            if not df.empty:
                price_columns = ("Price_baseline", "Price", "Price_Diff")
                for col in price_columns:
                    if col in df.columns:
                        # Convert column to numeric , then multiply by 100
                        df[col] = pd.to_numeric(df[col], errors='coerce') / 100
                data_frames[filename] = df
                consolidated_data.append(df)
            else:
                print(f"Warning: No valid data in file {filename}")
            pbar.update(n=1)
            

    # Consolidate all data into a single DataFrame
    if consolidated_data:
        consolidated_output = pd.concat(consolidated_data, ignore_index=True)
        try:
            # Perform any calculations 
            consolidated_output = calculate_consolidated_fields(consolidated_output)
            
            # Save the consolidated output
            consolidated_output_path = os.path.join(output_directory, "consolidated_output.xlsx")
            with pd.ExcelWriter(consolidated_output_path, engine='openpyxl') as writer:
                consolidated_output.to_excel(writer, sheet_name="Consolidated_Output", index=False)

                workbook = writer.book
                sheet = writer.sheets["Consolidated_Output"]
                
                
                # Header formating 
                for cell in sheet[1]:
                    cell.font = Font(bold=False)
                # column formatting
                for col in sheet.columns:
                    column_letter = col[0].column_letter
                    # Get the maximum length of the column header and its values
                    header_length = len(str(col[0].value)) if col[0].value else 0
                    # Calculate the column width
                    column_width = max(header_length + 0.5, 13)
                    # Set the column width
                    sheet.column_dimensions[column_letter].width = column_width

            print(f"Consolidated output saved at: {consolidated_output_path}")

        except Exception as e:
            print(f"Error during consolidated calculations or saving: {e}")
    else:
        print("No data to consolidate.")

    # Define the new output file name
    comparison_filename = "combined_output.xlsx"
    comparison_path = os.path.join(output_directory, comparison_filename)
    
    # Save comparison outputs to Excel
    baseline_df = data_frames.pop(baseline_file)
    with pd.ExcelWriter(comparison_path, engine='openpyxl') as writer:
        for key, df in data_frames.items():
            sheet_name = refine_suffix(key)  # Using the refined filename as sheet name
            if not sheet_name:
                print(f"Skipping sheet for file {key} due to invalid suffix.")
                continue  # Skip empty suffix
            
            # Align comparison DataFrame columns with combined_columns
            comparison_df = df.reindex(columns=[col.replace("_baseline", "") for col in combined_columns if "_baseline" not in col])

            # Combine baseline and comparison data
            combined_df = pd.concat(
                [baseline_df.add_suffix("_baseline"), comparison_df],
                axis=1
            ).reindex(columns=combined_columns)

            # Compute differences
            combined_df = compute_differences(combined_df)

            # Keep the difference columns
            difference_columns = [
                "Price_Diff", "Order_volume_Diff", "Item_Diff", 
                "Carton_volume_Diff", "volume_utilization_Diff", "total_weight_Diff"
            ]

            if combined_df.empty:
                print(f"Warning: Sheet {sheet_name} has no data. Skipping.")
                continue

            # Write the combined DataFrame to the corresponding sheet
            combined_df.to_excel(writer, sheet_name=sheet_name, index=False)

            # Apply formulas for Dimmed and Billed Weight
            workbook = writer.book
            sheet = writer.sheets[sheet_name]

            # Apply dynamic formatting
            apply_dynamic_column_formats(sheet)

            # Header formating 
            for cell in sheet[1]:
                cell.font = Font(bold=False)

            # Column width adjustments
            for col in sheet.columns:
                column_letter = col[0].column_letter
                header_length = len(str(col[0].value)) if col[0].value else 0
                column_width = max(header_length + 0.5, 13)
                sheet.column_dimensions[column_letter].width = column_width

            headers = [str(cell.value).strip() for cell in sheet[1]]
            dim_weight_col_idx = next((i + 1 for i, h in enumerate(headers) if h.startswith("dim_weight_")), None)
            total_weight_col_idx = next((i + 1 for i, h in enumerate(headers) if h.startswith("total_weight_")), None)

            if dim_weight_col_idx is None or total_weight_col_idx is None:
                print(f"Warning: Required columns for Dimmed and Billed Weight formulas not found in '{sheet_name}'. Headers: {headers}")
                continue

            last_column = len(headers)
            dimmed_col_letter = get_column_letter(last_column + 1)
            billed_weight_col_letter = get_column_letter(last_column + 2)

            sheet[f"{dimmed_col_letter}1"] = "Dimmed"
            sheet[f"{billed_weight_col_letter}1"] = "Billed_Weight"

            for row in range(2, sheet.max_row + 1):
                sheet[f"{dimmed_col_letter}{row}"] = (
                    f"=IF({get_column_letter(dim_weight_col_idx)}{row} > {get_column_letter(total_weight_col_idx)}{row}, \"Yes\", \"No\")"
                )
                sheet[f"{billed_weight_col_letter}{row}"] = (
                    f"=IF({get_column_letter(dim_weight_col_idx)}{row} > {get_column_letter(total_weight_col_idx)}{row}, "
                    f"ROUNDUP({get_column_letter(dim_weight_col_idx)}{row}, 0), ROUNDUP({get_column_letter(total_weight_col_idx)}{row}, 0))"
                )

            final_df = combined_df[difference_columns]
            
    print(f"Comparison Excel saved at {comparison_path}")

if __name__ == "__main__":
    directory = "Upload Path"
    output_directory = "Output Path"
    os.makedirs(output_directory, exist_ok=True)
    process_files(directory, output_directory)

In [ ]:
#Step 2
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from openpyxl import load_workbook

# Define the path to the Excel file
excel_file = "Output Path/consolidated_output.xlsx"
output_dir = os.path.abspath(os.path.dirname(excel_file))

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Custom color palette
custom_colors = {
    "gray": "#6a6a6b",
    "blue": "#0400f9",
    "seafoam": "#00ff80",
    "bright_purple": "#8100fb",
    "sky_blue": "#0A56B1",
    "green": "#307f2f",
    "brick": "#7f432f",
    "red": "#ff0000",
    "orange": "#ee6200"
}

# Function to format y-axis
format_thousands = lambda x, _: f'{int(x):,}'

# Function to group carton types
def group_carton_types(carton_name):
    if "Box" in carton_name:
        if "Small" in carton_name:
            return "Small Boxes"
        elif "Medium" in carton_name:
            return "Medium Boxes"
        elif "Large" in carton_name:
            return "Large Boxes"
    elif "Mailer" in carton_name:
        return "Mailers"
    else:
        return "Other"

# Read Excel sheets
excel_data = pd.ExcelFile(excel_file)
sheets = excel_data.sheet_names

# Generate plots for each sheet
for sheet in sheets:
    df = pd.read_excel(excel_file, sheet_name=sheet)

    # Group carton types
    if 'name' in df.columns:
        df['grouped_name'] = df['name'].apply(group_carton_types)
    
    # Convert data types
    df['surface_area'] = pd.to_numeric(df['surface_area'], errors='coerce')
    df['Carton_volume'] = pd.to_numeric(df['Carton_volume'], errors='coerce')

    # Filter missing values
    df = df.dropna(subset=['total_weight', 'Billed_Weight', 'dim_weight', 'Price', 'base_cost', 'Order_volume', 'Carton_volume'])

    # Summarize data
    summary_data = df.groupby('source_flag').agg({
        'Billed_Weight': 'sum',
        'dim_weight': 'sum',
        'total_weight': 'sum'
    }).reset_index()

        # Combo Chart: Dim Weight vs Total Weight
    if 'dim_weight' in df.columns and 'total_weight' in df.columns:
        plt.figure(figsize=(10, 6))
        sns.barplot(data=summary_data, x='source_flag', y='dim_weight', color= custom_colors['gray'])
        sns.lineplot(data=summary_data, x='source_flag', y=summary_data['total_weight'] / 2, color= custom_colors['seafoam'], marker='X', markersize=15, linewidth=3)
        plt.title('Combo Chart: Dim Weight vs Total Weight')
        plt.xlabel('Source Name')
        plt.ylabel('Dim Weight')
        plt.xticks(rotation=45)
        plt.gca().yaxis.set_major_formatter(FuncFormatter(format_thousands))
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "actual_vs_dim_weight.png"))
        plt.close()
    
        # Combo Chart: Billed Weight vs Total Weight
    if 'Billed_Weight' in df.columns and 'total_weight' in df.columns:
        plt.figure(figsize=(10, 6))
        sns.barplot(data=summary_data, x='source_flag', y='Billed_Weight', color= custom_colors['gray'])
        sns.lineplot(data=summary_data, x='source_flag', y=summary_data['total_weight'] / 2, color= custom_colors['brick'], marker='X', markersize=15, linewidth=3)
        plt.title('Combo Chart: Billed Weight vs Total Weight')
        plt.xlabel('Source Name')
        plt.ylabel('Billed Weight')
        plt.xticks(rotation=45)
        plt.gca().yaxis.set_major_formatter(FuncFormatter(format_thousands))
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "actual_vs_billed_weight.png"))
        plt.close()
    
        # Surface Area Chart
    if 'surface_area' in df.columns:
        surface_data = df.groupby('source_flag')['surface_area'].sum().reset_index()
        plt.figure(figsize=(12, 8))
        sns.barplot(data=surface_data, x='source_flag', y='surface_area',  hue='source_flag', legend=False)
        plt.title('Surface Area')
        plt.xlabel('Source Name')
        plt.ylabel('Total Surface Area')
        plt.xticks(rotation=45)
        plt.gca().yaxis.set_major_formatter(FuncFormatter(format_thousands))
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "SA_agg_comp.png"))
        plt.close()

     # Dimmed vs Count Chart
    if 'Dimmed' in df.columns:
        dimmed_data = df.groupby(['source_flag', 'Dimmed']).size().reset_index(name='count')
        plt.figure(figsize=(10, 6))
        sns.barplot(data=dimmed_data, x='source_flag', y='count', hue='Dimmed')
        plt.title('Counts of Dimmed (Yes/No) by Source Flag')
        plt.xlabel('Source Flag')
        plt.ylabel('Count')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "dimmed_vs_count.png"))
        plt.close()
    
    # Dim Weight vs Billed Weight
    if 'dim_weight' in df.columns and 'Billed_Weight' in df.columns:
        weight_data = df.groupby('source_flag').agg({'dim_weight': 'sum', 'Billed_Weight': 'sum'}).reset_index()
        plt.figure(figsize=(10, 6))
        sns.barplot(data=weight_data, x='source_flag', y='Billed_Weight', label='Billed Weight', color= custom_colors['gray'])
        sns.lineplot(data=weight_data, x='source_flag', y='dim_weight', marker='X', linewidth=3, markersize=15, color= custom_colors['red'], label='Dim Weight')
        plt.title('Sum of Dim Weight and Billed Weight by Source Flag')
        plt.xlabel('Source Flag')
        plt.ylabel('Total Weight')
        plt.legend(loc='upper right')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "dim_weight_vs_billed_weight.png"))
        plt.close()

    # Orders Billed Over Actual
    if 'Billed_over_Actual' in df.columns and 'orderId' in df.columns:
        orders_data = df.groupby(['Billed_over_Actual', 'source_flag']).size().reset_index(name='order_count')
        plt.figure(figsize=(10, 6))
        sns.lineplot(data=orders_data, x='Billed_over_Actual', y='order_count', hue='source_flag', marker='X', linewidth=5, markersize=15)
        plt.title('Combo Chart: Billed over Actual vs Count of Order ID by Source Flag')
        plt.xlabel('Billed over Actual')
        plt.ylabel('Count of Order ID')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "orders_billed_over_actual.png"))
        plt.close()

    # Price by Carton Type
    if 'Price' in df.columns:
        price_data = df.groupby(['name', 'source_flag']).agg({'Price': 'mean'}).reset_index()
        plt.figure(figsize=(12, 6))
        sns.barplot(data=price_data, x='name', y='Price', hue='source_flag')
        plt.title('Average Price / Carton Type')
        plt.xlabel('Carton Type')
        plt.ylabel('Average Price')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "price_by_carton_type.png"))
        plt.close()

    # Volume Utilization by Carton Type
    if 'volume_utilization' in df.columns:
        volume_data = df.groupby(['name', 'source_flag']).agg({'volume_utilization': 'mean'}).reset_index()
        plt.figure(figsize=(12, 6))
        sns.barplot(data=volume_data, x='name', y='volume_utilization', hue='source_flag')
        plt.legend(title='Source Flag', loc='upper right', bbox_to_anchor=(1.15, 1))
        plt.title('Average Volume Utilization / Carton Type')
        plt.xlabel('Carton Type')
        plt.ylabel('Average Volume Utilization')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "volume_utilization_by_carton_type.png"))
        plt.close()

print("All plots have been successfully generated and saved.")

In [ ]:
import os
import re
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from tqdm import tqdm
from PIL import Image as PILImage  
from io import BytesIO

# Function to open the image for insertion into Excel
def open_image(image_path, max_width=1200, max_height=1000):
    try:
        img = PILImage.open(image_path)  # Open the image using PIL
        img_width, img_height = img.size
        scale = min(max_width / img_width, max_height / img_height) # Calculate the resizing scale to maintain aspect ratio
        new_width = int(img_width * scale) # Calculate new dimensions
        new_height = int(img_height * scale) 
        img = img.resize((new_width, new_height), PILImage.Resampling.LANCZOS) # Resize the image
        img_byte_arr = BytesIO() # Convert the resized image to a byte stream (in memory) for use in openpyxl
        img.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)  # Reset pointer to the beginning of the byte stream
        return Image(img_byte_arr) # Return the image for insertion into Excel
    except FileNotFoundError:
        print(f"File {image_path} not found.")
        return None
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
        return None

# Main processing function to load and process files
def process_files(directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    files = os.listdir(directory)
    baseline_file = None
    for filename in files:
        if "baseline" in filename.lower():
            baseline_file = filename
            break

    if not baseline_file:
        raise ValueError("Baseline file not found. Ensure a file containing 'baseline' in its name exists.")

    image_directory = 'Output Path' 
    image_paths = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if f.endswith('.png')]
    
    wb = load_workbook('Output Path/combined_output.xlsx')  # Load the Excel file
    ws_images = wb.create_sheet('Visuals')  
    current_row = 2  # Initialize row position
    current_column = 2
    
    with tqdm(total=len(image_paths), desc="Inserting Images into Excel", unit="image", colour="cyan") as pbar:
        for i, image_path in enumerate(image_paths):
            img = open_image(image_path, max_width=1200, max_height=1000)

            if img:
                cell_position = f"{chr(64 + current_column)}{current_row}"
                ws_images.add_image(img, cell_position)
                # Adjust row and column for the next image
                current_column += 50  # Move to the next column
                if current_column > 1:  # Adjust number of columns per row 
                    current_column = 2  # Reset to column D
                    current_row += 50  # Move to the next row
            else:
                print(f"Image {image_path} could not be opened.")  # Debugging log
            pbar.update(1)
    
    run_match = re.search(r"filepath_(\d+)", baseline_file)  # Extracting run number 
    run_number = run_match.group(1)

    final_filename = f"_Output_run_{run_number}.xlsx" # Define the new output file name
    final_path = os.path.join(output_directory, final_filename)

    wb.save(final_path)  
    print(f"Final file saved with visuals on a separate sheet: {final_filename}")

if __name__ == "__main__":
    directory = "Upload Path"  
    output_directory = "Output Path"  
    process_files(directory, output_directory)

In [ ]:
import re

def refine_suffix(filename):

    match = re.search(r"filepath_(\d+)\.(.*?)(?:$|\.)", filename)
    if match:
        return match.group(2)
    return None

# Test
print(refine_suffix("filepath_####.baseline.output_cartons")) 
print(refine_suffix("filepath_####.fileA.output_cartons"))
print(refine_suffix("filepath_####.fileB.output_cartons"))